In [ ]:
from datetime import datetime, timedelta, timezone
import threading
import time
import os

import pandas as pd
import requests

from mainnet_launch.database.schema.full import Blocks
from mainnet_launch.database.postgres_operations import (
    insert_avoid_conflicts,
    get_subset_not_already_in_column,
    _exec_sql_and_cache,
)
from mainnet_launch.data_fetching.get_state_by_block import get_raw_state_by_blocks, build_blocks_to_use
from mainnet_launch.data_fetching.defi_llama.fetch_timestamp import get_block_by_timestamp_defi_llama

from mainnet_launch.constants import *
from mainnet_launch.data_fetching.defi_llama.fetch_timestamp import fetch_blocks_by_unix_timestamps_defillama


def _determine_missing_timestamps(chain:ChainData) -> list[int]:
    """
    
    We know that we have the top block if a given day 
    iff, the largest block of that day + 1 == smallest block of the next day.

    and both blocks are in our database.
    
    """
    query = f"""
    select 
        date(datetime AT TIME ZONE 'UTC') as date,  
        max(block) as largest_block_today, 
        min(block) as smallest_block_today,
        extract(epoch from date(datetime AT TIME ZONE 'UTC'))::bigint as unix_timestamp
    from blocks
    where chain_id = {chain.chain_id}
    group by date(datetime AT TIME ZONE 'UTC')
    order by date(datetime AT TIME ZONE 'UTC') asc
    """

    df = _exec_sql_and_cache(query)
    df["smallest_block_tommorrow"] = df["smallest_block_today"].shift(-1).astype("Int64")

    df["we_know_the_top_block_of_today"] = df["largest_block_today"] + 1 == df["smallest_block_tommorrow"]

    zeroth_second_of_each_day_we_need_to_get = df[~df["we_know_the_top_block_of_today"]]["unix_timestamp"].to_list()
    return zeroth_second_of_each_day_we_need_to_get




/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/eth_abi/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Fetching 3rd-party data from https://coins.llama.fi/block/ethereum/1724976000: 100%|██████████| 10/10 [00:01<00:00,  7.73it/s]
